## AutoML
## Empirical Tests - TPOT

This project aims to explore some of the main **AutoML tools** available, which involves the following tasks:
1. Reading of technical articles concerning the automated machine learning field.
2. Discussion about machine learning pipelines and the automation of some of their components.
3. Identification of the most interesting Python libraries for automatic ML pipeline construction.
4. Quick implementation of the selected tools with simulated data.
5. Careful exploration of the APIs of the selected tools.
6. Comparison among selected tools concerning: model performance, computation time, and usability.

All of these activities derive from the **objectives** of this project, which are: i) reflection about ML pipeline components; ii) discussion and analysis of AutoML tools; iii) identification of key-points of AutoML frameworks; iv) definition of: the advantages and disadvantages of main AutoML tools, and, first of all, the relavance and adequacy of implementing AutoML.

---------------------

In this series of notebooks, we test out different AutoML Python libraries and compare them according to the following criteria: performance metrics of developed pipelines evaluated on test data; computation time (i.e., the performance relative to the available time budget of the search process); and usability of the tool.

* **Performance:** for each tool, after providing them with a training data (that will receive the appropriate validation approach by each tool), and after the search for the best ML pipeline, the selected one will be evaluated on a hold-out dataset (25% of the complete dataset). The model assessment will be based on the following metrics, since the supervised learning task is a binary classification here: [ROC-AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), [average precision score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html), [Brier score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html), [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), and [MCC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html).

* **Computation time:** all tested AutoML tools have some sort of time budget for the search process. Therefore, instead of minimizing the computation time across all tested tools, we will explore three different time budgets: 20 minutes, 1 hour, and 6 hours. Consequently, one of the main aspects of the comparison among tools will be the performance achieved by each one of them given different time budgets, besides of the average performance throughout all time budgets.

* **Usability:** this aspect of the comparison refers to how easy it is to set up the search for each one of the tested tools. Also important are the outputs of the search process, mainly in terms of the visualization and assessment of constructed and selected pipelines. Besides, the diversity of produced information about the search and how clear it is to access and interpret these data are also an aspect to have in mind. Finally, the more straightforward it is to use a selected pipeline the better is the tool.

The empirical tests follow the reading and discussing of the APIs of all selected tools. So, since the main initialization arguments, methods and attributes have been defined, they will be used accordingly in these notebooks.

The data used for the empirical tests was found in Kaggle repository of datasets. It consists of a dataset for binary classification whose objective is to construct a classification algorithm for the [identification of malware apps](https://www.kaggle.com/saurabhshahane/android-permission-dataset). It has 27310 unique instances (mobile phone applications) and 184 variables, among which one is the binary outcome variable and another is the name of the app. Since the main objective of this project is to explore AutoML tools, only some basic feature engineering operations were implemented, besides of a short description and exploration of the data.

------------

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Importing datasets](#imports)<a href='#imports'></a>.
    * [Features and outcome variables](#feats_outcomes)<a href='#feats_outcomes'></a>.


5. [Data description](#data_description)<a href='#data_description'></a>.
6. [ML pipeline](#ml_pipeline)<a href='#ml_pipeline'></a>.

<a id='libraries'></a>

## Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "/content/gdrive/MyDrive/Studies/autoML/Codes"

/content/gdrive/MyDrive/Studies/autoML/Codes


In [3]:
# pip install tpot

In [4]:
# pip install -r requirements.txt

In [5]:
# pip uninstall scikit-learn

In [6]:
# pip install scikit-learn==0.24.2

In [7]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from time import time
import pickle

from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score, brier_score_loss, accuracy_score, matthews_corrcoef
from sklearn.model_selection import RepeatedStratifiedKFold

# pip install tpot
import tpot # TPOT
from tpot import TPOTClassifier
print(f'TPOT: version {tpot.__version__}.')

TPOT: version 0.11.7.


<a id='functions_classes'></a>

## Functions and classes

In [8]:
from utils import running_time, correct_col_name, train_test_split
from pre_process import pre_process

<a id='settings'></a>

## Settings

### Data management

In [9]:
# Identification of the test:
estimation_id = str(int(time()))

# Declare whether to export results:
export = True

### ML pipeline search

#### Search complexity parameters

In [10]:
# Time budget (minutes) for the algorithm to finally select the best pipeline:
max_time_mins = 6*60

# Time budget (minutes) for each tested pipeline:
max_eval_time_mins = 5 if max_time_mins==20 else (30 if max_time_mins==6*60 else 20)

#### Estimation parameters

In [11]:
# Cross-validation strategy for assessing tested pipelines:
cv = 5

# Performance metric optimized during the search:
scoring = 'roc_auc'

# Random fraction of training data used at each iteration:
subsample = 0.75

#### Computation parameters

In [12]:
# Number of processes to use in parallel:
n_jobs = -1

<a id='imports'></a>

## Importing datasets

<a id='feats_outcomes'></a>

### Features and outcome variables

In [13]:
# Importing data:
df = pd.read_csv('../Datasets/Android_Permission.csv')

# Columns names:
df.columns = [correct_col_name(c) for c in df.columns]

# Auxiliary variables:
drop_vars = ['app', 'package', 'class']

# Removing duplicates:
df.drop_duplicates(inplace=True)

print(f'Shape of data: {df.shape}.')
df.head(3)

Shape of data: (27310, 184).


,app,package,category,description,rating,number_of_ratings,price,related_apps,dangerous_permissions_count,safe_permissions_count,access_drm_content_,access_email_provider_data,access_all_system_downloads,access_download_manager_,advanced_download_manager_functions_,audio_file_access,install_drm_content_,modify_google_service_configuration,modify_google_settings,move_application_resources,read_google_settings,send_download_notifications_,voice_search_shortcuts,access_surfaceflinger,access_checkin_properties,access_the_cache_filesystem,access_to_passwords_for_google_accounts,act_as_an_account_authenticator,bind_to_a_wallpaper,bind_to_an_input_method,change_screen_orientation,coarse,control_location_update_notifications,control_system_backup_and_restore,delete_applications,delete_other_applications_caches,delete_other_applications_data,directly_call_any_phone_numbers,directly_install_applications,disable_or_modify_status_bar,...,your_accounts_access_other_google_services,your_accounts_act_as_an_account_authenticator,your_accounts_act_as_the_accountmanagerservice,your_accounts_contacts_data_in_google_accounts,your_accounts_discover_known_accounts,your_accounts_manage_the_accounts_list,your_accounts_read_google_service_configuration,your_accounts_use_the_authentication_credentials_of_an_account,your_accounts_view_configured_accounts,your_location_access_extra_location_provider_commands,your_location_coarse,your_location_fine,your_location_mock_location_sources_for_testing,your_messages_read_email_attachments,your_messages_send_gmail,your_messages_edit_sms_or_mms,your_messages_modify_gmail,your_messages_read_gmail,your_messages_read_gmail_attachment_previews,your_messages_read_sms_or_mms,your_messages_read_instant_messages,your_messages_receive_mms,your_messages_receive_sms,your_messages_receive_wap,your_messages_send_sms_received_broadcast,your_messages_send_wap_push_received_broadcast,your_messages_write_instant_messages,your_personal_information_add_or_modify_calendar_events_and_send_email_to_guests,your_personal_information_choose_widgets,your_personal_information_read_browsers_history_and_bookmarks,your_personal_information_read_calendar_events,your_personal_information_read_contact_data,your_personal_information_read_sensitive_log_data,your_personal_information_read_user_defined_dictionary,your_personal_information_retrieve_system_internal_state,your_personal_information_set_alarm_in_alarm_clock,your_personal_information_write_browsers_history_and_bookmarks,your_personal_information_write_contact_data,your_personal_information_write_to_user_defined_dictionary,class
0,Canada Post Corporation,com.canadapost.android,Business,Canada Post Mobile App gives you access to som...,3.1,77,0.0,"{com.adaffix.pub.ca.android, com.kevinquan.gas...",7.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,Word Farm,com.realcasualgames.words,Brain & Puzzle,Speed and strategy combine in this exciting wo...,4.3,199,0.0,"{air.com.zubawing.FastWordLite, com.joybits.do...",3.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fortunes of War FREE,fortunesofwar.free,Cards & Casino,"Fortunes of War is a fast-paced, easy to learn...",4.1,243,0.0,"{com.kevinquan.condado, hu.monsta.pazaak, net....",1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Features names

In [14]:
features_names = list(df.drop(drop_vars, axis=1).columns)

for i in range(len(features_names)):
  print(features_names[i:i+10], '\n')

['category', 'description', 'rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data'] 

['description', 'rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads'] 

['rating', 'number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads', 'access_download_manager_'] 

['number_of_ratings', 'price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_', 'access_email_provider_data', 'access_all_system_downloads', 'access_download_manager_', 'advanced_download_manager_functions_'] 

['price', 'related_apps', 'dangerous_permissions_count', 'safe_permissions_count', 'access_drm_content_

#### Data types

In [15]:
data_types = pd.DataFrame(df.dtypes, columns=['type']).reset_index(drop=False)
data_types.columns = ['feature', 'type']

print('\033[1mDistribution of data types:\033[0m')
print(data_types.type.value_counts())

Distribution of data types:
int64      176
object       5
float64      3
Name: type, dtype: int64


#### Train-test split

In [16]:
df_train, df_test = train_test_split(df, test_ratio=0.25, shuffle=True)

#### Feature engineering

Related apps

In [17]:
# Creating the variable with the number of related apps:
df_train['related_apps'] = df_train['related_apps'].apply(lambda x: x if pd.isna(x) else x.replace('{', '').replace('}', ''))
df_train['num_related_apps'] = df_train.related_apps.apply(lambda x: x if pd.isna(x) else len(x.split(',')))
df_test['num_related_apps'] = df_test.related_apps.apply(lambda x: x if pd.isna(x) else len(x.split(',')))

# Updating the list of auxiliary variables:
drop_vars.append('related_apps')

Description

In [18]:
# Creating the variable that indicates the number of words in a description:
df_train['num_words_desc'] = df_train.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))
df_test['num_words_desc'] = df_test.description.apply(lambda x: x if pd.isna(x) else len(x.split(' ')))

# Updating the list of auxiliary variables:
drop_vars.append('description')

Category

Even though this feature engineering is actually a transformation applied over categorical features, we first implement one-hot encoding in order to translate this categorical attribute into a numerical one, since some AutoML tools explored within this project do not allow textual inputs.

In [19]:
from transformations import applying_one_hot

In [20]:
transf_data = applying_one_hot(training_data=df_train, cat_vars=['category'], variance_param=-1, test_data=df_test)
df_train = transf_data['training_data']
df_test = transf_data['test_data']

Number of categorical features: 1
Number of overall selected dummies: 30.


<a id='data_description'></a>

## Data description

<a id='features_types'></a>

### Features types

In [21]:
feature_types = pd.DataFrame(df_train.drop(drop_vars, axis=1).dtypes, columns=['type']).reset_index(drop=False)
feature_types.columns = ['feature', 'type']

print('\033[1mDistribution of data types (features):\033[0m')
print(feature_types.type.value_counts())

Distribution of data types (features):
int64      175
uint8       30
float64      5
Name: type, dtype: int64


<a id='ml_pipeline'></a>

## ML pipeline

In [22]:
help(TPOTClassifier)

Help on class TPOTClassifier in module tpot.tpot:

class TPOTClassifier(tpot.base.TPOTBase)
 |  TPOTClassifier(generations=100, population_size=100, offspring_size=None, mutation_rate=0.9, crossover_rate=0.1, scoring=None, cv=5, subsample=1.0, n_jobs=1, max_time_mins=None, max_eval_time_mins=5, random_state=None, config_dict=None, template=None, warm_start=False, memory=None, use_dask=False, periodic_checkpoint_folder=None, early_stop=None, verbosity=0, disable_update_check=False, log_file=None)
 |  
 |  TPOT estimator for classification problems.
 |  
 |  Method resolution order:
 |      TPOTClassifier
 |      tpot.base.TPOTBase
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  classification = True
 |  
 |  default_config_dict = {'sklearn.cluster.FeatureAgglomeration': {'affin...
 |  
 |  regression = False
 |  
 |  scoring_function = 'accuracy'
 |  
 |  ----------------------------------------------------------------

<a id='ml_pipeline_search'></a>

### ML pipeline search

#### Setting the search

In [23]:
# Creating the AutoML object:
model = TPOTClassifier(
    # Search complexity parameters:
    max_time_mins=max_time_mins, max_eval_time_mins=max_eval_time_mins,

    # Estimation parameters:
    cv=cv, scoring=scoring, subsample=subsample,
    
    # Computation parameters:
    n_jobs=n_jobs, random_state=1, verbosity=2
)

#### Running the search

In [24]:
start_time = datetime.now()

# Running the search:
model.fit(df_train.drop(drop_vars, axis=1), df_train['class'])

# Exporting the ML pipeline chosen after the search:
model.export(f'../Datasets/Outcomes/tpot/best_pipeline_{estimation_id}.py')

# Total elapsed time:
end_time = datetime.now()
search_time = running_time(start_time=start_time, end_time=end_time)

Imputing missing values in feature set


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.900742163718933

Generation 2 - Current best internal CV score: 0.900742163718933

Generation 3 - Current best internal CV score: 0.9011348957130579

370.02 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.1, max_depth=9, min_child_weight=11, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0)
------------------------------------
Running time: 370.68 minutes.
Start time: 2021-09-07, 13:02:47
End time: 2021-09-07, 19:13:28
------------------------------------


### Assessing the outcomes

#### ML pipeline

Constructed pipelines

In [25]:
# Dictionary with the pipelines tested during the search:
ml_pipelines = model.evaluated_individuals_

print(f'Number of tested pipelines: {len(ml_pipelines)}.')

Number of tested pipelines: 471.


In [26]:
# Dictionary with the Pareto-efficient pipelines tested during the search:
efficient_ml_pipelines = model.pareto_front_fitted_pipelines_

print(f'Number of Pareto-efficient pipelines: {len(efficient_ml_pipelines)}.')

Number of Pareto-efficient pipelines: 1.


Final pipeline

In [27]:
# Selected ML pipeline:
print(model.fitted_pipeline_)

Pipeline(steps=[('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.1,
                               max_delta_step=0, max_depth=9,
                               min_child_weight=11, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=1, num_parallel_tree=1, random_state=1,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1.0, tree_method='exact',
                               validate_parameters=1, verbosity=0))])


In [28]:
# Python module with the selected pipeline:
with open('../Datasets/Outcomes/tpot/best_pipeline.py', 'r') as file:
  lines = file.readlines()

print('\033[1mBest ML pipeline:\033[0m')
print(''.join(lines))

Best ML pipeline:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=1)

imputer = SimpleImputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.9011021228183301
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    XGBCl

#### Model evaluation

In [29]:
# Predictions for hold-out data:
y_hat = [p[1] for p in model.predict_proba(df_test.drop(drop_vars, axis=1))]

# Performance metrics of the best model:
test_roc_auc = roc_auc_score(df_test['class'], y_hat)

test_avg_prec = average_precision_score(df_test['class'], y_hat)
test_brier = brier_score_loss(df_test['class'], y_hat)
test_acc = accuracy_score(df_test['class'], [1 if p > 0.5 else 0 for p in y_hat])
test_mcc = matthews_corrcoef(df_test['class'], [1 if p > 0.5 else 0 for p in y_hat])

print(f'Test ROC-AUC: {test_roc_auc:.4f}.')
print(f'Test average-precision score: {test_avg_prec:.4f}.')
print(f'Test Brier score: {test_brier:.4f}.')
print(f'Test accuracy: {test_acc:.4f}.')
print(f'Test MCC: {test_mcc:.4f}.')

Imputing missing values in feature set
Test ROC-AUC: 0.8985.
Test average-precision score: 0.9554.
Test Brier score: 0.1207.
Test accuracy: 0.8116.
Test MCC: 0.5831.


In [30]:
# More direct presentation of metric evaluated on test set:
print(f'{model.scoring} = {model.score(df_test.drop(drop_vars, axis=1), df_test["class"]):.4f}')

Imputing missing values in feature set
roc_auc = 0.8985


#### Model assessment

In [31]:
model_assess = {
    "estimation_id": str(estimation_id),
    "autoML": "tpot",
    "parameters": {
      "search_complexity": {
        "max_time_mins": max_time_mins, "max_eval_time_mins": max_eval_time_mins
      },
      "estimation": {
        "cv": cv, "scoring": scoring, "subsample": subsample
      },
      "computation": {
        "n_jobs": n_jobs
      }
    },
    "running_time": search_time,
    "performance_metrics": {
        "test_roc_auc": test_roc_auc, "test_avg_prec": test_avg_prec, "test_brier": test_brier, "test_acc": test_acc, "test_mcc": test_mcc
    }
}

### Exporting the outcomes

In [32]:
if export:
  # Dictionary with the pipelines tested during the search:
  with open(f'../Datasets/Outcomes/tpot/ml_pipelines_{estimation_id}.json', 'w') as json_file:
    json.dump(ml_pipelines, json_file, indent=2)

  # Selected ML pipelines:
  with open(f'../Datasets/Outcomes/tpot/selected_pipeline_{estimation_id}.json', 'w') as json_file:
    json.dump({'selected_pipeline': str(model.fitted_pipeline_)}, json_file, indent=2)

  # ML pipeline:
  # pickle.dump(model, open(f'../Datasets/Outcomes/tpot/model_{estimation_id}.pickle', 'wb'))

  # Model assessment:
  with open(f'../Datasets/Outcomes/tpot/model_assess_{estimation_id}.json', 'w') as json_file:
    json.dump(model_assess, json_file, indent=2)